In [138]:
!pip install dask[complete]
! pip install pmdarima

     |████████████████████████████████| 592 kB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 15.6 MB 10.9 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

tensorflow 2.3.1 requires numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.5 which is incompatible.
You should consider upgrading via the '/Users/pedrogodeiro/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [140]:
! pip uninstall statsmodels -y

Found existing installation: statsmodels 0.11.1
Uninstalling statsmodels-0.11.1:
  Successfully uninstalled statsmodels-0.11.1


In [6]:
! pip install statsmodels==0.12.2

     |████████████████████████████████| 9.5 MB 387 kB/s eta 0:00:01
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.11.0
    Uninstalling statsmodels-0.11.0:
      Successfully uninstalled statsmodels-0.11.0
You should consider upgrading via the '/Users/pedrogodeiro/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [147]:
! pip uninstall numpy -y

Found existing installation: numpy 1.18.0
Uninstalling numpy-1.18.0:
  Successfully uninstalled numpy-1.18.0


In [150]:
! pip install numpy==1.19.0

  Using cached numpy-1.19.0-cp38-cp38-macosx_10_9_x86_64.whl (15.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.3
    Uninstalling numpy-1.20.3:
      Successfully uninstalled numpy-1.20.3
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

tensorflow 2.3.1 requires numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.0 which is incompatible.
You should consider upgrading via the '/Users/pedrogodeiro/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [5]:
! pip install statsmodels

You should consider upgrading via the '/Users/pedrogodeiro/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [13]:
import dask.dataframe as dd
import datetime
from collections import defaultdict
import numpy as np
import pandas as pd
from pmdarima.arima import auto_arima
from pmdarima.arima import ADFTest
from sklearn.metrics import r2_score
from statsmodels.tsa.forecasting.theta import ThetaModel
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
import warnings
import time

In [2]:
warnings.filterwarnings("ignore")

In [114]:
df = dd.read_csv('df_total.csv')
df.head()

,u,v,Family,Category,Brand,Name,week,sku,store
0,1.0,1945.0,21,132,142,277,2016-01-01,1011,334
1,1.0,675.0,21,110,143,282,2016-01-01,1030,69
2,1.0,3249.0,21,35,145,288,2016-01-01,1045,52
3,1.0,1040.0,21,127,147,294,2016-01-01,1052,4
4,1.0,1640.0,21,127,147,296,2016-01-01,1053,77


In [77]:
df.week=df.week.astype("category")
df.week=df.week.cat.as_known()

In [78]:
df['yr'] = df.week.astype(str).str[:4]
df['yr'] = df['yr'].astype(int)

In [6]:
df.Family.value_counts().compute()

12    19456378
9     18904590
1     15854221
21    12707831
15     5184114
2      4726479
18     3371660
14     2467194
8      1573372
16     1384912
10     1072013
6      1027194
3       915883
5       593242
13      510118
7       484548
11      352505
20      198715
4       132641
19      129248
17      114071
Name: Family, dtype: int64

In [87]:
df.store.value_counts().compute()

282    487491
383    455758
103    438786
359    429227
272    417931
        ...  
221     86301
167     76883
220     74814
247     73934
243     66557
Name: store, Length: 410, dtype: int64

In [7]:
df.Category.value_counts().compute()

178    63120299
175     3336120
22      2489541
3       2295849
27      2050406
         ...   
176           6
136           4
77            4
68            2
90            1
Name: Category, Length: 178, dtype: int64

In [8]:
df.Brand.value_counts().compute()

1425    1262885
1410    1112067
230      974359
133      961364
487      939737
         ...   
1359          1
648           1
157           1
664           1
678           1
Name: Brand, Length: 1523, dtype: int64

In [9]:
df.Name.value_counts().compute()

2609    901309
567     836639
2802    789120
2662    775567
1422    741801
         ...  
2332         1
581          1
2344         1
2345         1
1423         1
Name: Name, Length: 2820, dtype: int64

In [10]:
df.sku.value_counts().compute()

3008    487569
1572    474773
3835    464774
3696    462508
3362    460374
         ...  
2280         1
2281         1
3716         1
2285         1
2351         1
Name: sku, Length: 8509, dtype: int64

# Loop Name

In [98]:
df_name_week=pd.read_csv("df_fcst_name.csv")
df_name_week=df_name_week.drop(labels=0, axis=0)
names=df_name_week.drop(['week_start'], axis=1).columns.to_list()

In [154]:
df_name_week

,week_start,1,2,3,4,5,6,7,8,9,...,2842,2843,2844,2845,2846,2847,2848,2849,2850,yr
1,2016-01-04,0.0,4.0,101.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,13.0,2120.0,1589.0,7667.5,79.0,72.0,46.0,2016
2,2016-01-11,0.0,5.0,79.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,2297.0,1412.0,7866.0,67.0,78.0,38.0,2016
3,2016-01-18,0.0,6.0,102.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,9.0,2206.0,1246.0,7281.0,70.0,51.0,52.0,2016
4,2016-01-25,0.0,1.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,7.0,2075.0,1134.0,6830.0,88.0,55.0,52.0,2016
5,2016-02-01,0.0,2.0,96.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,9.0,2121.0,1157.0,6805.0,83.0,53.0,42.0,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,2019-09-30,5.0,5.0,21.0,20.0,11.0,16.0,34.0,31.0,27.0,...,0.0,0.0,0.0,1808.0,3196.0,6644.0,70.0,47.0,19.0,2019
197,2019-10-07,3.0,5.0,17.0,18.0,5.0,13.0,25.0,56.0,21.0,...,0.0,0.0,0.0,2105.0,3334.0,7983.0,71.0,62.0,28.0,2019
198,2019-10-14,3.0,7.0,15.0,5.0,3.0,7.0,45.0,82.0,22.0,...,0.0,0.0,0.0,2093.0,3162.0,7094.0,64.0,54.0,32.0,2019
199,2019-10-21,5.0,5.0,22.0,5.0,10.0,6.0,57.0,89.0,24.0,...,0.0,0.0,0.0,1857.0,3111.0,6721.0,50.0,56.0,35.0,2019


In [157]:
dates=["2019-11-04","2019-11-11","2019-11-18","2019-11-25","2019-12-02","2019-12-09","2019-12-16","2019-12-23","2019-12-30"]
df3=pd.DataFrame(columns=names, index=dates)

In [99]:
%%time
df_name_week['yr'] = df_name_week.week_start.astype(str).str[:4]
df_name_week['yr'] = df_name_week['yr'].astype(int)
name_train = df_name_week[53:181]
name_test = df_name_week[182:200]
name_train.index = pd.to_datetime(name_train.index)
name_train=name_train.set_index("week_start")
name_test.index = pd.to_datetime(name_test.index)
name_test=name_test.set_index("week_start")
models=pd.DataFrame()
product_name=[]
official_score=[]
model_selected=[]
for i in names:
    name_train_one=name_train[i]
    name_train_one=pd.Series(name_train_one)
    name_test_one=name_test[i]
    name_test_one=pd.Series(name_test_one)
    #predictions=pd.DataFrame(arima_model.predict(n_periods=18),index=name_test_one.index)
    #predictions.columns=["predicted_sales"]
    name_train_one.index = pd.to_datetime(name_train_one.index)
    tm = ThetaModel(name_train_one)
    res = tm.fit()
    model = ETSModel(name_train_one, error="add", trend="add", seasonal="add",damped_trend=True, seasonal_periods=52)
    fit = model.fit()
    pred = fit.get_prediction(start='2019-07-01', end='2019-10-28')
    df = pred.summary_frame(alpha=0.05)
    predictions_theta=res.forecast(18)
    predictions_theta.index=name_test_one.index
    predictions=pd.DataFrame(predictions_theta,index=name_test_one.index)
    predictions["test"]=name_test_one
    predictions["predictions_ets"]=df["mean"]
    r2_theta=r2_score(predictions["test"],predictions["forecast"])
    r2_ets=r2_score(predictions["test"],predictions["predictions_ets"])
    product_name.append(i)
    if r2_theta>0.13 or r2_ets>0.13:
        if r2_theta > r2_ets:
            official_score.append(r2_theta)
            model_selected.append("Theta Model")
        if r2_ets > r2_theta:
            official_score.append(r2_ets)
            model_selected.append("ETS Model")
    else:
        official_score.append("Not Enough")
        model_selected.append("None")
models["Name"]=product_name
models["Best Score"]=official_score
models["Best Model"]=model_selected
models

CPU times: user 1h 2min 36s, sys: 3min 6s, total: 1h 5min 43s
Wall time: 57min 47s


,Name,Best Score,Best Model
0,1,Not Enough,None
1,2,Not Enough,None
2,3,Not Enough,None
3,4,0.872902,Theta Model
4,5,Not Enough,None
...,...,...,...
2815,2846,Not Enough,None
2816,2847,0.70828,Theta Model
2817,2848,Not Enough,None
2818,2849,Not Enough,None


In [100]:
models["Best Model"].value_counts()

None           2234
ETS Model       460
Theta Model     126
Name: Best Model, dtype: int64

# Loop Store

In [101]:
df_store_week=pd.read_csv("df_fcst_store.csv")
df_store_week=df_store_week.drop(labels=0, axis=0)
stores=df_store_week.drop(['week_start'], axis=1).columns.to_list()

In [102]:
%%time
df_store_week['yr'] = df_store_week.week_start.astype(str).str[:4]
df_store_week['yr'] = df_store_week['yr'].astype(int)
store_train = df_store_week[53:181]
store_test = df_store_week[182:200]
store_train.index = pd.to_datetime(store_train.index)
store_train=store_train.set_index("week_start")
store_test.index = pd.to_datetime(store_test.index)
store_test=store_test.set_index("week_start")
models_store=pd.DataFrame()
store_id=[]
official_score=[]
model_selected=[]
for i in stores:
    store_train_one=store_train[i]
    store_train_one=pd.Series(store_train_one)
    store_test_one=store_test[i]
    store_test_one=pd.Series(store_test_one)
    store_train_one.index = pd.to_datetime(store_train_one.index)
    tm = ThetaModel(store_train_one)
    res = tm.fit()
    model = ETSModel(store_train_one, error="add", trend="add", seasonal="add",damped_trend=True, seasonal_periods=52)
    fit = model.fit()
    pred = fit.get_prediction(start='2019-07-01', end='2019-10-28')
    df = pred.summary_frame(alpha=0.05)
    predictions_theta=res.forecast(18)
    predictions_theta.index=store_test_one.index
    predictions=pd.DataFrame(predictions_theta,index=store_test_one.index)
    predictions["test"]=store_test_one
    predictions["predictions_ets"]=df["mean"]
    r2_theta=r2_score(predictions["test"],predictions["forecast"])
    r2_ets=r2_score(predictions["test"],predictions["predictions_ets"])
    store_id.append(i)
    if r2_theta>0.13 or r2_ets>0.13:
        if r2_theta > r2_ets:
            official_score.append(r2_theta)
            model_selected.append("Theta Model")
        if r2_ets > r2_theta:
            official_score.append(r2_ets)
            model_selected.append("ETS Model")
    else:
        official_score.append("Not Enough")
        model_selected.append("None")
models_store["Store ID"]=store_id
models_store["Best Score"]=official_score
models_store["Best Model"]=model_selected
models_store

CPU times: user 8min 15s, sys: 26.8 s, total: 8min 41s
Wall time: 5min 28s


,Store ID,Best Score,Best Model
0,1,0.568665,Theta Model
1,2,0.562149,Theta Model
2,3,0.375777,Theta Model
3,4,0.25057,Theta Model
4,5,0.329557,Theta Model
...,...,...,...
405,406,0.383117,Theta Model
406,407,0.635111,ETS Model
407,408,0.525474,ETS Model
408,409,0.490147,ETS Model


In [103]:
models_store["Best Model"].value_counts()

Theta Model    196
ETS Model      148
None            66
Name: Best Model, dtype: int64

# Loop Brand

In [104]:
df_brand_week=pd.read_csv("df_fcst_brand.csv")
df_brand_week = df_brand_week[df_brand_week.week_start != "2015-12-28"]
df_brand_week=df_brand_week.pivot_table(values='  u', index='week_start', columns='Brand', aggfunc='sum')
df_brand_week=df_brand_week.fillna(0.0)
df_brand_week=df_brand_week.reset_index()
brands=df_brand_week.drop(['week_start'], axis=1).columns.to_list()

In [105]:
%%time
df_brand_week['yr'] = df_brand_week.week_start.astype(str).str[:4]
df_brand_week['yr'] = df_brand_week['yr'].astype(int)
brand_train = df_brand_week[53:181]
brand_test = df_brand_week[182:200]
brand_train.index = pd.to_datetime(brand_train.index)
brand_train=brand_train.set_index("week_start")
brand_test.index = pd.to_datetime(brand_test.index)
brand_test=brand_test.set_index("week_start")
models_brand=pd.DataFrame()
brand_id=[]
official_score=[]
model_selected=[]
for i in brands:
    brand_train_one=brand_train[i]
    brand_train_one=pd.Series(brand_train_one)
    brand_test_one=brand_test[i]
    brand_test_one=pd.Series(brand_test_one)
    brand_train_one.index = pd.to_datetime(brand_train_one.index)
    tm = ThetaModel(brand_train_one)
    res = tm.fit()
    model = ETSModel(brand_train_one, error="add", trend="add", seasonal="add",damped_trend=True, seasonal_periods=52)
    fit = model.fit()
    pred = fit.get_prediction(start='2019-07-01', end='2019-10-28')
    df = pred.summary_frame(alpha=0.05)
    predictions_theta=res.forecast(18)
    predictions_theta.index=brand_test_one.index
    predictions=pd.DataFrame(predictions_theta,index=brand_test_one.index)
    predictions["test"]=brand_test_one
    predictions["predictions_ets"]=df["mean"]
    r2_theta=r2_score(predictions["test"],predictions["forecast"])
    r2_ets=r2_score(predictions["test"],predictions["predictions_ets"])
    brand_id.append(i)
    if r2_theta>0.13 or r2_ets>0.13:
        if r2_theta > r2_ets:
            official_score.append(r2_theta)
            model_selected.append("Theta Model")
        if r2_ets > r2_theta:
            official_score.append(r2_ets)
            model_selected.append("ETS Model")
    else:
        official_score.append("Not Enough")
        model_selected.append("None")
models_brand["brand ID"]=brand_id
models_brand["Best Score"]=official_score
models_brand["Best Model"]=model_selected
models_brand

CPU times: user 34min 33s, sys: 1min 42s, total: 36min 15s
Wall time: 24min 15s


,brand ID,Best Score,Best Model
0,1,0.535344,Theta Model
1,2,Not Enough,None
2,3,Not Enough,None
3,4,0.755621,ETS Model
4,5,0.396687,ETS Model
...,...,...,...
1518,1531,0.668793,Theta Model
1519,1532,Not Enough,None
1520,1533,0.70828,Theta Model
1521,1534,Not Enough,None


In [106]:
models_brand["Best Model"].value_counts()

None           1114
ETS Model       313
Theta Model      96
Name: Best Model, dtype: int64

# Loop Category

In [107]:
df_category_week=pd.read_csv("df_fcst_category.csv")
df_category_week = df_category_week[df_category_week.week_start != "2015-12-28"]
df_category_week=df_category_week.pivot_table(values='  u', index='week_start', columns='Category', aggfunc='sum')
df_category_week=df_category_week.fillna(0.0)
df_category_week=df_category_week.reset_index()
categorys=df_category_week.drop(['week_start'], axis=1).columns.to_list()

In [108]:
%%time
df_category_week['yr'] = df_category_week.week_start.astype(str).str[:4]
df_category_week['yr'] = df_category_week['yr'].astype(int)
category_train = df_category_week[53:181]
category_test = df_category_week[182:200]
category_train.index = pd.to_datetime(category_train.index)
category_train=category_train.set_index("week_start")
category_test.index = pd.to_datetime(category_test.index)
category_test=category_test.set_index("week_start")
models_category=pd.DataFrame()
category_id=[]
official_score=[]
model_selected=[]
for i in categorys:
    category_train_one=category_train[i]
    category_train_one=pd.Series(category_train_one)
    category_test_one=category_test[i]
    category_test_one=pd.Series(category_test_one)
    category_train_one.index = pd.to_datetime(category_train_one.index)
    tm = ThetaModel(category_train_one)
    res = tm.fit()
    model = ETSModel(category_train_one, error="add", trend="add", seasonal="add",damped_trend=True, seasonal_periods=52)
    fit = model.fit()
    pred = fit.get_prediction(start='2019-07-01', end='2019-10-28')
    df = pred.summary_frame(alpha=0.05)
    predictions_theta=res.forecast(18)
    predictions_theta.index=category_test_one.index
    predictions=pd.DataFrame(predictions_theta,index=category_test_one.index)
    predictions["test"]=category_test_one
    predictions["predictions_ets"]=df["mean"]
    r2_theta=r2_score(predictions["test"],predictions["forecast"])
    r2_ets=r2_score(predictions["test"],predictions["predictions_ets"])
    category_id.append(i)
    if r2_theta>0.13 or r2_ets>0.13:
        if r2_theta > r2_ets:
            official_score.append(r2_theta)
            model_selected.append("Theta Model")
        if r2_ets > r2_theta:
            official_score.append(r2_ets)
            model_selected.append("ETS Model")
    else:
        official_score.append("Not Enough")
        model_selected.append("None")
models_category["category ID"]=category_id
models_category["Best Score"]=official_score
models_category["Best Model"]=model_selected
models_category

CPU times: user 4min 14s, sys: 11.9 s, total: 4min 26s
Wall time: 3min 2s


,category ID,Best Score,Best Model
0,1,0.861194,Theta Model
1,2,0.887422,Theta Model
2,3,0.792618,Theta Model
3,4,0.752205,Theta Model
4,5,0.806228,ETS Model
...,...,...,...
173,174,Not Enough,None
174,175,0.708693,Theta Model
175,176,Not Enough,None
176,177,Not Enough,None


In [112]:
models_category["Best Model"].value_counts()

None           113
ETS Model       33
Theta Model     32
Name: Best Model, dtype: int64

# Loop Family

In [109]:
df_family_week=pd.read_csv("df_fcst_family.csv")
df_family_week = df_family_week[df_family_week.week_start != "2015-12-28"]
df_family_week=df_family_week.pivot_table(values='  u', index='week_start', columns='Family', aggfunc='sum')
df_family_week=df_family_week.fillna(0.0)
df_family_week=df_family_week.reset_index()
familys=df_family_week.drop(['week_start'], axis=1).columns.to_list()

In [110]:
%%time
df_family_week['yr'] = df_family_week.week_start.astype(str).str[:4]
df_family_week['yr'] = df_family_week['yr'].astype(int)
family_train = df_family_week[53:181]
family_test = df_family_week[182:200]
family_train.index = pd.to_datetime(family_train.index)
family_train=family_train.set_index("week_start")
family_test.index = pd.to_datetime(family_test.index)
family_test=family_test.set_index("week_start")
models_family=pd.DataFrame()
family_id=[]
official_score=[]
model_selected=[]
for i in familys:
    family_train_one=family_train[i]
    family_train_one=pd.Series(family_train_one)
    family_test_one=family_test[i]
    family_test_one=pd.Series(family_test_one)
    family_train_one.index = pd.to_datetime(family_train_one.index)
    tm = ThetaModel(family_train_one)
    res = tm.fit()
    model = ETSModel(family_train_one, error="add", trend="add", seasonal="add",damped_trend=True, seasonal_periods=52)
    fit = model.fit()
    pred = fit.get_prediction(start='2019-07-01', end='2019-10-28')
    df = pred.summary_frame(alpha=0.05)
    predictions_theta=res.forecast(18)
    predictions_theta.index=family_test_one.index
    predictions=pd.DataFrame(predictions_theta,index=family_test_one.index)
    predictions["test"]=family_test_one
    predictions["predictions_ets"]=df["mean"]
    r2_theta=r2_score(predictions["test"],predictions["forecast"])
    r2_ets=r2_score(predictions["test"],predictions["predictions_ets"])
    family_id.append(i)
    if r2_theta>0.13 or r2_ets>0.13:
        if r2_theta > r2_ets:
            official_score.append(r2_theta)
            model_selected.append("Theta Model")
        if r2_ets > r2_theta:
            official_score.append(r2_ets)
            model_selected.append("ETS Model")
    else:
        official_score.append("Not Enough")
        model_selected.append("None")
models_family["family ID"]=family_id
models_family["Best Score"]=official_score
models_family["Best Model"]=model_selected
models_family

CPU times: user 22 s, sys: 1.3 s, total: 23.3 s
Wall time: 13.6 s


,family ID,Best Score,Best Model
0,1,0.703161,ETS Model
1,2,0.742779,ETS Model
2,3,0.384297,ETS Model
3,4,0.466702,ETS Model
4,5,0.650152,ETS Model
5,6,0.444414,Theta Model
6,7,0.712893,Theta Model
7,8,0.518609,ETS Model
8,9,0.732885,ETS Model
9,10,0.485966,ETS Model


In [111]:
models_family["Best Model"].value_counts()

ETS Model      11
Theta Model     8
None            2
Name: Best Model, dtype: int64

# Name Dictionary

In [127]:
predictions_final=pd.DataFrame()
names=df.Name.unique().compute()
predictions_final["Name"]=names

In [122]:
predictions_final=predictions_final.set_index("Name")

In [141]:
df2=df[["Family","Category","Brand","Name"]].compute()

In [145]:
df2=df2.drop_duplicates(subset="Name")

In [147]:
df2=df2.set_index("Name")

In [248]:
df2.to_csv("name dictionary.csv")

# Predictions Name

In [222]:
dates=["2019-11-04","2019-11-11","2019-11-18","2019-11-25","2019-12-02","2019-12-09","2019-12-16","2019-12-23","2019-12-30"]
df_name_predictions=pd.DataFrame(columns=names, index=dates)
models=models.set_index("Name")
name_train = df_name_week[53:200]
name_train.index = pd.to_datetime(name_train.index)
name_train=name_train.set_index("week_start")
for i in names:
    name_train_one=name_train[str(i)]
    name_train_one=pd.Series(name_train_one)
    name_train_one.index = pd.to_datetime(name_train_one.index)
    if models.loc[str(i)][1]=="Theta Model":
        tm = ThetaModel(name_train_one)
        res = tm.fit()
        predictions_theta=res.forecast(9)
        predictions_theta.index=dates
        df_name_predictions[i]=predictions_theta
    if models.loc[str(i)][1]=="ETS Model":
        model = ETSModel(name_train_one, error="add", trend="add", seasonal="add",damped_trend=True, seasonal_periods=52)
        fit = model.fit()
        pred = fit.get_prediction(start='2019-11-04', end='2019-12-30')
        df_name_predictions[i]=pred
        df = pred.summary_frame(alpha=0.05)
        df_name_predictions[i]=df["mean"]       

In [225]:
df_name_predictions

Name,277,282,288,294,296,297,300,299,305,306,...,833,2325,755,1267,560,2352,2541,10,2539,2666
2019-11-04,NaN,NaN,NaN,NaN,NaN,233.401863,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-11,NaN,NaN,NaN,NaN,NaN,222.208167,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-18,NaN,NaN,NaN,NaN,NaN,171.315199,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-25,NaN,NaN,NaN,NaN,NaN,183.688841,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-02,NaN,NaN,NaN,NaN,NaN,169.579183,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-09,NaN,NaN,NaN,NaN,NaN,192.905110,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-16,NaN,NaN,NaN,NaN,NaN,126.825124,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-23,NaN,NaN,NaN,NaN,NaN,126.959562,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-30,NaN,NaN,NaN,NaN,NaN,135.121358,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [247]:
df_name_predictions.to_csv("name_predictions.csv")

# Predictions Store

In [228]:
models_store

,Store ID,Best Score,Best Model
0,1,0.568665,Theta Model
1,2,0.562149,Theta Model
2,3,0.375777,Theta Model
3,4,0.25057,Theta Model
4,5,0.329557,Theta Model
...,...,...,...
405,406,0.383117,Theta Model
406,407,0.635111,ETS Model
407,408,0.525474,ETS Model
408,409,0.490147,ETS Model


In [229]:
dates=["2019-11-04","2019-11-11","2019-11-18","2019-11-25","2019-12-02","2019-12-09","2019-12-16","2019-12-23","2019-12-30"]
df_store_predictions=pd.DataFrame(columns=stores, index=dates)
models_store=models_store.set_index("Store ID")
store_train = df_store_week[53:200]
store_train.index = pd.to_datetime(store_train.index)
store_train=store_train.set_index("week_start")
for i in stores:
    store_train_one=store_train[str(i)]
    store_train_one=pd.Series(store_train_one)
    store_train_one.index = pd.to_datetime(store_train_one.index)
    if models_store.loc[str(i)][1]=="Theta Model":
        tm = ThetaModel(store_train_one)
        res = tm.fit()
        predictions_theta=res.forecast(9)
        predictions_theta.index=dates
        df_store_predictions[i]=predictions_theta
    if models_store.loc[str(i)][1]=="ETS Model":
        model = ETSModel(store_train_one, error="add", trend="add", seasonal="add",damped_trend=True, seasonal_periods=52)
        fit = model.fit()
        pred = fit.get_prediction(start='2019-11-04', end='2019-12-30')
        df_store_predictions[i]=pred
        df = pred.summary_frame(alpha=0.05)
        df_store_predictions[i]=df["mean"]

In [230]:
df_store_predictions

,1,2,3,4,5,6,7,8,9,10,...,401,402,403,404,405,406,407,408,409,410
2019-11-04,3111.361859,2311.318735,3461.341703,4552.959030,2297.388157,2114.937501,1690.231647,NaN,2815.451216,1430.313672,...,3224.967096,1858.737017,1477.163335,4368.186460,3716.651059,2546.935646,4661.331959,2720.908630,2513.528509,4108.870235
2019-11-11,3025.416135,2013.396366,3257.138794,4454.864062,2267.218100,2023.871305,1738.891866,NaN,2422.481382,1457.029046,...,3010.045262,1902.418670,1523.255399,4410.311615,4273.689856,2668.872816,4054.936327,2491.349667,2439.122533,4067.064971
2019-11-18,2966.886149,2092.115504,3283.707952,4128.430672,2278.120492,1931.565813,1530.183881,NaN,2261.694766,1380.963602,...,2939.446071,1677.787080,1233.157869,3845.423899,3333.495593,2204.748015,4417.668196,2654.323167,2458.570094,4110.490637
2019-11-25,3295.826124,2182.645534,3274.837611,4410.257251,2247.750287,2042.226222,1497.540231,NaN,2657.273387,1451.018000,...,2648.114807,2122.840651,1592.168136,4202.563536,3826.168336,2716.517001,3738.389370,2222.049048,1921.682477,4502.125666
2019-12-02,3609.428708,2603.625015,3795.464126,5045.557306,2651.014200,2514.150938,1624.478276,NaN,2995.745461,1792.253369,...,3164.968141,2231.563181,1689.944627,4783.438198,4085.752551,2818.978582,4383.885548,2582.226586,2305.087420,4558.632713
2019-12-09,3973.044160,2546.862244,3679.051971,5283.967423,2668.848566,2363.175998,1799.466939,NaN,2901.056491,1567.015554,...,3213.778541,2226.629372,1732.527990,4601.844508,4245.484488,2731.192908,4673.132316,2701.612809,2906.528143,4676.137684
2019-12-16,3242.446042,2154.823267,3466.660560,4486.876399,2369.120938,1919.755705,1976.957806,NaN,2314.763572,1392.341762,...,3445.537266,1552.095995,1088.887890,3270.641135,3412.645208,2010.961866,4541.501940,2742.011532,2587.973486,3937.285220
2019-12-23,3205.589362,2144.741608,3702.161430,4659.314341,2456.737621,2087.313252,1823.764773,NaN,2768.365247,1463.344219,...,2540.325814,1930.984496,1461.830346,3720.147402,3548.028031,2435.682347,3170.864243,1999.375334,1865.619356,4127.843832
2019-12-30,3833.218046,2817.462232,4115.004429,5209.189845,2936.900544,2479.262381,1856.941239,NaN,2980.234125,1826.847198,...,2990.057588,2364.266021,1859.482041,4932.778978,4231.026619,2895.323775,3703.385905,2244.069458,2072.997250,4958.493583


In [244]:
df_store_predictions.to_csv("store_predictions.csv")

# Predictions Brand

In [232]:
brand_train

Brand,1,2,3,4,5,6,7,8,9,10,...,1527,1528,1529,1530,1531,1532,1533,1534,1535,yr
week_start,,,,,,,,,,,,,,,,,,,,,
2017-01-09,94.0,0.0,0.0,2282.0,11372.0,5.0,0.0,367.0,0.0,2978.0,...,1450.0,0.0,0.0,0.0,2324.0,2035.0,7765.0,76.0,103.0,2017
2017-01-16,88.0,0.0,0.0,2063.0,10768.0,9.0,0.0,339.0,0.0,2309.0,...,1338.0,0.0,0.0,1.0,2138.0,1930.0,6813.0,67.0,90.0,2017
2017-01-23,102.0,0.0,0.0,1971.0,10501.0,8.0,0.0,366.0,0.0,2075.0,...,1325.0,0.0,0.0,0.0,2105.0,1931.0,6461.0,67.0,95.0,2017
2017-01-30,73.0,0.0,0.0,2166.0,10131.0,4.0,0.0,420.0,0.0,1961.0,...,1371.0,0.0,0.0,0.0,2161.0,1911.0,6274.0,67.0,99.0,2017
2017-02-06,91.0,0.0,0.0,2186.0,11476.0,1.0,0.0,357.0,0.0,1796.0,...,1498.0,0.0,0.0,0.0,2335.0,1924.0,6945.0,94.0,77.0,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-09-30,113.0,0.0,0.0,2795.0,8670.0,7.0,731.0,526.0,115.0,862.0,...,2251.0,0.0,0.0,0.0,1808.0,3196.0,6644.0,70.0,66.0,2019
2019-10-07,92.0,3.0,0.0,3116.0,10192.0,5.0,880.0,684.0,162.0,1076.0,...,2494.0,0.0,0.0,0.0,2105.0,3334.0,7983.0,71.0,90.0,2019
2019-10-14,84.0,2.0,0.0,2915.0,8791.0,8.0,884.0,616.0,200.0,1101.0,...,2334.0,0.0,0.0,0.0,2093.0,3162.0,7094.0,64.0,86.0,2019


In [233]:
models_brand

,Best Score,Best Model
brand ID,,
1,0.535344,Theta Model
2,Not Enough,None
3,Not Enough,None
4,0.755621,ETS Model
5,0.396687,ETS Model
...,...,...
1531,0.668793,Theta Model
1532,Not Enough,None
1533,0.70828,Theta Model


In [236]:
dates=["2019-11-04","2019-11-11","2019-11-18","2019-11-25","2019-12-02","2019-12-09","2019-12-16","2019-12-23","2019-12-30"]
df_brand_predictions=pd.DataFrame(columns=brands, index=dates)
#models_brand=models_brand.set_index("brand ID")
brand_train = df_brand_week[53:200]
brand_train.index = pd.to_datetime(brand_train.index)
brand_train=brand_train.set_index("week_start")
for i in brands:
    brand_train_one=brand_train[i]
    brand_train_one=pd.Series(brand_train_one)
    brand_train_one.index = pd.to_datetime(brand_train_one.index)
    if models_brand.loc[i][1]=="Theta Model":
        tm = ThetaModel(brand_train_one)
        res = tm.fit()
        predictions_theta=res.forecast(9)
        predictions_theta.index=dates
        df_brand_predictions[i]=predictions_theta
    if models_brand.loc[i][1]=="ETS Model":
        model = ETSModel(brand_train_one, error="add", trend="add", seasonal="add",damped_trend=True, seasonal_periods=52)
        fit = model.fit()
        pred = fit.get_prediction(start='2019-11-04', end='2019-12-30')
        df_brand_predictions[i]=pred
        df = pred.summary_frame(alpha=0.05)
        df_brand_predictions[i]=df["mean"]
df_brand_predictions

In [237]:
df_brand_predictions

,1,2,3,4,5,6,7,8,9,10,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
2019-11-04,68.533895,NaN,NaN,2769.149935,9630.350758,NaN,773.246071,NaN,NaN,946.890813,...,NaN,NaN,NaN,NaN,NaN,1963.513192,NaN,7393.149408,NaN,NaN
2019-11-11,66.144843,NaN,NaN,2596.088202,9193.358760,NaN,659.290696,NaN,NaN,970.746432,...,NaN,NaN,NaN,NaN,NaN,1863.080888,NaN,7064.330500,NaN,NaN
2019-11-18,58.689456,NaN,NaN,2504.589373,8600.832461,NaN,579.015996,NaN,NaN,887.237552,...,NaN,NaN,NaN,NaN,NaN,1663.998803,NaN,6207.969167,NaN,NaN
2019-11-25,50.727031,NaN,NaN,2420.502159,8417.711675,NaN,418.991745,NaN,NaN,1106.846030,...,NaN,NaN,NaN,NaN,NaN,1794.949608,NaN,7016.082656,NaN,NaN
2019-12-02,61.791334,NaN,NaN,2404.404908,9041.603164,NaN,356.889520,NaN,NaN,1304.665474,...,NaN,NaN,NaN,NaN,NaN,2036.751243,NaN,8538.965034,NaN,NaN
2019-12-09,85.468929,NaN,NaN,2464.288852,10797.455632,NaN,420.111802,NaN,NaN,1430.905176,...,NaN,NaN,NaN,NaN,NaN,1680.155890,NaN,7587.961490,NaN,NaN
2019-12-16,54.496836,NaN,NaN,2302.395762,9310.286950,NaN,378.214969,NaN,NaN,1573.111328,...,NaN,NaN,NaN,NaN,NaN,1154.459882,NaN,5494.390516,NaN,NaN
2019-12-23,66.902893,NaN,NaN,1922.674869,6178.815675,NaN,335.494780,NaN,NaN,1733.740244,...,NaN,NaN,NaN,NaN,NaN,1298.315676,NaN,5827.524174,NaN,NaN
2019-12-30,64.138461,NaN,NaN,2539.919974,6622.325352,NaN,386.499149,NaN,NaN,1859.615028,...,NaN,NaN,NaN,NaN,NaN,1865.182368,NaN,7900.529780,NaN,NaN


In [243]:
df_brand_predictions.to_csv("brand_predictions.csv")

# Predictions Category

In [240]:
models_category

,category ID,Best Score,Best Model
0,1,0.861194,Theta Model
1,2,0.887422,Theta Model
2,3,0.792618,Theta Model
3,4,0.752205,Theta Model
4,5,0.806228,ETS Model
...,...,...,...
173,174,Not Enough,None
174,175,0.708693,Theta Model
175,176,Not Enough,None
176,177,Not Enough,None


In [241]:
dates=["2019-11-04","2019-11-11","2019-11-18","2019-11-25","2019-12-02","2019-12-09","2019-12-16","2019-12-23","2019-12-30"]
df_category_predictions=pd.DataFrame(columns=categorys, index=dates)
models_category=models_category.set_index("category ID")
category_train = df_category_week[53:200]
category_train.index = pd.to_datetime(category_train.index)
category_train=category_train.set_index("week_start")
for i in categorys:
    category_train_one=category_train[i]
    category_train_one=pd.Series(category_train_one)
    category_train_one.index = pd.to_datetime(category_train_one.index)
    if models_category.loc[i][1]=="Theta Model":
        tm = ThetaModel(category_train_one)
        res = tm.fit()
        predictions_theta=res.forecast(9)
        predictions_theta.index=dates
        df_category_predictions[i]=predictions_theta
    if models_category.loc[i][1]=="ETS Model":
        model = ETSModel(category_train_one, error="add", trend="add", seasonal="add",damped_trend=True, seasonal_periods=52)
        fit = model.fit()
        pred = fit.get_prediction(start='2019-11-04', end='2019-12-30')
        df_category_predictions[i]=pred
        df = pred.summary_frame(alpha=0.05)
        df_category_predictions[i]=df["mean"]
df_category_predictions

,1,2,3,4,5,6,7,8,9,10,...,169,170,171,172,173,174,175,176,177,178
2019-11-04,5868.798374,19567.632087,18963.129433,1588.379093,200.131220,NaN,3246.574778,NaN,1776.474822,5462.285505,...,NaN,NaN,NaN,NaN,NaN,NaN,53406.449454,NaN,NaN,830111.685137
2019-11-11,6284.177665,20481.276873,20442.479511,1669.082495,214.337776,NaN,3328.295721,NaN,1875.295783,5718.400585,...,NaN,NaN,NaN,NaN,NaN,NaN,51167.213942,NaN,NaN,797229.576874
2019-11-18,6200.863963,20119.322070,20824.199598,1607.662303,212.900910,NaN,3569.361768,NaN,1993.646346,5823.942734,...,NaN,NaN,NaN,NaN,NaN,NaN,47169.093743,NaN,NaN,768299.299579
2019-11-25,7196.514828,22810.511974,24200.901163,1824.969106,205.259017,NaN,3932.415425,NaN,2456.183044,6453.763950,...,NaN,NaN,NaN,NaN,NaN,NaN,51542.435043,NaN,NaN,711185.572386
2019-12-02,8703.870703,27814.929020,29162.580207,2202.749037,230.510858,NaN,4173.465667,NaN,2928.828669,6876.744280,...,NaN,NaN,NaN,NaN,NaN,NaN,59513.524636,NaN,NaN,752254.745935
2019-12-09,10049.194493,32057.406789,34450.096759,2503.330108,250.238645,NaN,5603.824767,NaN,3552.894049,7573.625879,...,NaN,NaN,NaN,NaN,NaN,NaN,54458.318962,NaN,NaN,899312.277079
2019-12-16,11716.416150,36726.475555,42042.544023,2690.801143,344.819917,NaN,4851.021567,NaN,4186.519302,7424.453199,...,NaN,NaN,NaN,NaN,NaN,NaN,38889.687478,NaN,NaN,798722.944503
2019-12-23,12701.085451,39640.446792,43358.210749,3038.394463,357.181055,NaN,3413.978882,NaN,4331.430710,7752.647588,...,NaN,NaN,NaN,NaN,NaN,NaN,41983.351331,NaN,NaN,569061.737452
2019-12-30,12620.024788,40217.164094,42710.380881,3209.478046,221.919693,NaN,4090.052566,NaN,4057.313197,8720.779075,...,NaN,NaN,NaN,NaN,NaN,NaN,56009.631026,NaN,NaN,623483.433489


In [245]:
df_category_predictions.to_csv("category_predictions.csv")

# Predictions Family

In [242]:
dates=["2019-11-04","2019-11-11","2019-11-18","2019-11-25","2019-12-02","2019-12-09","2019-12-16","2019-12-23","2019-12-30"]
df_family_predictions=pd.DataFrame(columns=familys, index=dates)
models_family=models_family.set_index("family ID")
family_train = df_family_week[53:200]
family_train.index = pd.to_datetime(family_train.index)
family_train=family_train.set_index("week_start")
for i in familys:
    family_train_one=family_train[i]
    family_train_one=pd.Series(family_train_one)
    family_train_one.index = pd.to_datetime(family_train_one.index)
    if models_family.loc[i][1]=="Theta Model":
        tm = ThetaModel(family_train_one)
        res = tm.fit()
        predictions_theta=res.forecast(9)
        predictions_theta.index=dates
        df_family_predictions[i]=predictions_theta
    if models_family.loc[i][1]=="ETS Model":
        model = ETSModel(family_train_one, error="add", trend="add", seasonal="add",damped_trend=True, seasonal_periods=52)
        fit = model.fit()
        pred = fit.get_prediction(start='2019-11-04', end='2019-12-30')
        df_family_predictions[i]=pred
        df = pred.summary_frame(alpha=0.05)
        df_family_predictions[i]=df["mean"]
df_family_predictions

,1,2,3,4,5,6,7,8,9,10,...,12,13,14,15,16,17,18,19,20,21
2019-11-04,182096.771889,79736.840263,27985.398883,1033.031975,9818.993131,16771.555926,6811.429167,38740.039013,263210.023195,7646.841408,...,209142.486453,4741.374647,63271.430082,48656.798417,10797.404614,1408.585182,57167.846331,NaN,2142.038695,75789.717468
2019-11-11,175800.995990,74905.241140,26753.837406,971.561061,9700.348208,16079.923089,6436.008122,37909.221835,252660.885048,7389.905524,...,200735.542840,4997.935354,64393.921634,51578.109361,11775.621738,1056.195190,54494.749039,NaN,2006.916590,78627.330383
2019-11-18,168091.248276,72511.378842,26683.359258,901.924757,8924.530012,15201.192480,5696.600384,35885.780076,241152.634892,7469.113795,...,197022.305054,5030.976926,68233.913434,51855.897412,12267.244661,891.675766,52493.091604,NaN,2195.297167,71719.840156
2019-11-25,156088.253137,65601.244750,24738.430126,792.555704,8523.651144,15274.616037,6373.533415,33703.270761,219443.111265,7016.418502,...,186386.772735,5919.028795,79188.544867,59351.813250,14649.974877,840.660032,48508.570226,NaN,2459.494383,74565.041607
2019-12-02,164022.012640,70300.661074,26363.346524,785.913904,8768.953158,16530.217264,7595.089462,36375.630363,234664.205647,7243.304289,...,194706.051255,6800.208129,85621.481317,71126.747111,17324.958744,658.705554,52317.624266,NaN,2443.418039,81659.858042
2019-12-09,196224.993979,88564.522193,30347.910936,808.575493,11458.337467,16910.900264,6836.244012,41678.393057,289411.518451,7642.298903,...,222096.835327,7815.762522,102957.891516,83800.396222,20829.560370,609.743478,62610.661479,NaN,2792.230783,91868.107766
2019-12-16,176607.854034,76258.984562,26886.457950,724.244233,9906.631962,12018.368216,4575.914042,37448.582248,249212.304373,7325.020013,...,204404.968665,8688.576178,118972.511029,96475.581835,24477.712642,374.530509,54969.119247,NaN,2823.054628,70818.905325
2019-12-23,130187.933322,53197.843644,21899.633826,456.352206,6602.184329,12446.756779,4883.033664,26327.263511,175552.999810,5752.352037,...,146548.850573,9818.757448,116480.517217,99893.104169,23707.986976,514.723075,37557.899411,NaN,3048.074280,71623.758431
2019-12-30,139734.693938,58263.854431,23456.674428,444.622844,7431.443355,16708.075390,6986.074721,28628.184450,193908.586829,6732.766214,...,162889.156902,11638.543637,117056.862262,100725.441812,23438.720195,1014.336097,41483.121269,NaN,3733.772894,82997.679414


In [246]:
df_family_predictions.to_csv("family_predictions.csv")